In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


In [ ]:
mnist.train.images[0].shape

In [ ]:
mnist.train.images[0][251:260]

In [ ]:
for i in range(100):
    plt.figure()
    plt.imshow(-mnist.train.images[i].reshape(28,28),interpolation='nearest', cmap='gray')
    plt.title(mnist.train.labels[i])

In [ ]:
import pywt

In [ ]:
data = mnist.train.images[0].reshape(28,28)
coeffs = pywt.wavedec2(data, 'haar')
for i in range(1,len(coeffs)):
    print i, coeffs[i][0].shape, coeffs[i][1].shape, coeffs[i][2].shape

In [ ]:
def init_coeffs_mnist(coeffs):
    coeffs_zero = [np.zeros((2,2))]
    for i in range(1,5):
        coeffs_zero += [[]]   
        for j in range(3) :
            s = np.shape(coeffs[i][j])        
            coeffs_zero[i] += [np.zeros(s)]        
    return coeffs_zero    

In [ ]:
def copy_coeffs_mnist(coeffs):
    coeffs_test = []
    coeffs_test += [coeffs[0].copy()]
    for i in range(1,5):
        coeffs_test += [[]]    
        for j in range(3) :
            coeffs_test[i] += [np.copy(coeffs[i][j])]
    return coeffs_test    

In [ ]:
def calcule_path(coord, axes):
    path_i = []
    path_j = []
    for h in range(0,5):
        h_opp = 5 - h
        coord_i = int(coord[0] / 2**h_opp) 
        coord_j = int(coord[1] / 2**h_opp)    
        path_i += [axes[h][coord_i]]
        path_j += [axes[h][coord_j]]
    return path_i, path_j

In [ ]:
coeffs_test = copy_coeffs_mnist(coeffs)
coeffs_zero = init_coeffs_mnist(coeffs)
        
plt.figure(figsize=(12,8))
plt.subplot(131)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

for j in range(3) :
    coeffs_test[4][j] *= 0
    coeffs_zero[4][j] = np.copy(coeffs[4][j])
    
plt.subplot(132)
plt.imshow(-pywt.waverec2(coeffs_test, 'haar'), interpolation='nearest', cmap='gray')

plt.subplot(133)
plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')

In [ ]:
axes = []
for h in range(0,5):
    h_opp = 5 - h
    b_moins = -.5
    b_plus = 27.5
    intervalle = b_plus - b_moins
    coord_max = int(26.5 / 2**h_opp) + 1
    unite = max(intervalle / coord_max, 2**h_opp)
    print unite
    coords = []
    for j in range(coord_max):
        coords += [b_moins + (j+.5) * unite]
    axes += [coords]
print axes

In [ ]:
U_ref = []
U = []
for h in range(len(axes)):
    U_ref += [[]]
    for i in axes[h]:
        for j in axes[h]:
            U_ref[h] += [(i,j)]
    U += [[]]
    for i in range(len(axes[h])):
        for j in range(len(axes[h])):
            U[h] += [(h,i,j)]    
    print U[h]
    print U_ref[h]


In [ ]:
#coord = (10.5,10.5)
#coord = (7.5,16.5)
coord = (16.5,4.5)

coeffs_coord = init_coeffs_mnist(coeffs)
coeffs_coord[0] = np.copy(coeffs[0])
for h in range(1,5):
    h_opp = 5 - h
    coord_i = int(coord[0] / 2**h_opp) 
    coord_j = int(coord[1] / 2**h_opp)
    print coord_i, coord_j
    for j in range(3) :
        coeffs_coord[h][j][coord_i,coord_j] = coeffs[h][j][coord_i,coord_j]
        
plt.figure(figsize=(20,12))

path_i, path_j = calcule_path(coord, axes)

plt.subplot(131)
plt.imshow(-pywt.waverec2(coeffs, 'haar'), interpolation='nearest', cmap='gray')
#for u in U_ref:
#    plt.plot(u[1],u[0],'+',color='b',markersize=12)
plt.plot(path_j,path_i,'r')
plt.plot(path_j,path_i,'r+',markersize=12)
plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
plt.xlim([b_moins,b_plus])
plt.ylim([b_moins,b_plus])
plt.gca().invert_yaxis()

plt.subplot(132)
plt.imshow(-pywt.waverec2(coeffs_coord, 'haar'), interpolation='nearest', cmap='gray')
plt.plot(path_j,path_i,'r')
plt.plot(path_j,path_i,'r+',markersize=12)
plt.plot(coord[1],coord[0],'+',color='r',markersize=12)
plt.xlim([b_moins,b_plus])
plt.ylim([b_moins,b_plus])
plt.gca().invert_yaxis()


## Creation de la base

In [ ]:
len(mnist.train.images)

In [ ]:
B = []
for i in range(len(mnist.train.images)):
    c = np.where(mnist.train.labels[i] ==1)[0][0]
    x = mnist.train.images[i].reshape(28,28)
    coeffs = pywt.wavedec2(x, 'haar')
    u0 = U[0][0]
    v0 = np.array([coeffs[0][0][1], coeffs[0][1][0], coeffs[0][1][1]])
    B += [(v0,(c,u0))]
    for h in range(1,5):
        for u in U[h]:
            v = np.array([coeffs[h][0][u[1]][u[2]], coeffs[h][1][u[1]][u[2]], coeffs[h][2][u[1]][u[2]]])
            #if np.linalg.norm(v) > 1e-16:
            B += [(v,(c,u))]

In [ ]:
len(B)

In [ ]:
B[2]

## Dictionnaire

In [ ]:
Data = [[],[],[],[],[],[],[],[],[],[]]
for c in range(10):
    Data[c] = [{},{},{},{},{}] 
    
for d in B:
    v = d[0]
    c = d[1][0]
    h = d[1][1][0]
    pos = (d[1][1][1],d[1][1][2])
    if pos in Data[c][h]:
        Data[c][h][pos] += [v]
    else:
        Data[c][h][pos] = [v]


In [ ]:
print np.mean(Data[0][0][(0,0)],0)
print np.mean(Data[0][1][(0,0)],0)
print np.mean(Data[0][1][(0,1)],0)
print np.mean(Data[0][2][(0,0)],0)
print np.mean(Data[0][2][(0,1)],0)


In [ ]:
c = 1
h = 0
i = 0
j = 0
len(Data[c][h][(i,j)])

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure(figsize = (12,12))
ax = fig.add_subplot(111, projection='3d')

h = 4
i = 5
j = 5

test = np.array(Data[0][h][(i,j)])
ax.scatter(test[:,2], test[:,1], test[:,2],c='r', marker = 'o')

test2 = np.array(Data[1][h][(i,j)])
ax.scatter(test2[:,0], test2[:,1], test2[:,2],c='b', marker = '^')

test3 = np.array(Data[3][h][(i,j)])
ax.scatter(test3[:,0], test3[:,1], test3[:,2],c='g', marker = 'd')

ax.view_init(20, -30)

'''for angle in range(0, 360):
    ax.view_init(30, angle)
    plt.draw()
    plt.pause(.001)'''

## Probas elementaires 

In [ ]:
mu = {}
Sigma = {}
for c in range(10):
    for h in range(5):
        for u in U[h]:
            u_ref = (u[1],u[2])
            if u_ref in Data[c][h]:
                if len(Data[c][h][u_ref]) > 1:
                    mu[(c,h,u_ref)] = np.mean(Data[c][h][u_ref],0)
                    Sigma[(c,h,u_ref)] = np.cov(np.array(Data[c][h][u_ref]).T)
            

In [ ]:
c = 0
h = 4
u = (0,0)
print mu[(c,h,u)]
print Sigma[(c,h,u)]

In [ ]:
np.random.randn(3)

In [ ]:
## Reconstruction
for c in range(10):
    coeffs_zero = init_coeffs_mnist(coeffs)

    coeffs_zero[0][0,1]= mu[(c,0,(0,0))][0]
    coeffs_zero[0][1,0] = mu[(c,0,(0,0))][1]
    coeffs_zero[0][1,1] = mu[(c,0,(0,0))][2]
    for h in range(1,5):
        for u in U[h]:
            u_ref = (u[1],u[2])
            coeffs_zero[h][0][u[1]][u[2]] = mu[(c,h,u_ref)][0]

    plt.figure(figsize=(12,8))
    plt.subplot(131)
    plt.imshow(-pywt.waverec2(coeffs_zero, 'haar'), interpolation='nearest', cmap='gray')

